<a href="https://colab.research.google.com/github/SmithTheGreat/FPLAI/blob/main/FPLAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 71.2 MB/s eta 0:00:00


In [3]:
import requests
import pandas as pd
import numpy as np
import pulp
import math

In [ ]:
def get_top_5_whitelist():
    data = requests.get("https://fantasy.premierleague.com/api/bootstrap-static/").json()
    elements = pd.DataFrame(data['elements'])
    top5 = elements.sort_values('total_points', ascending=False).head(5)
    return set(top5['web_name'].tolist())

WHITELIST = get_top_5_whitelist()
print("Whitelist (top 5 scorers 24-25):", WHITELIST)

def fetch_fpl_data():
    url = "https://fantasy.premierleague.com/api/bootstrap-static/"
    return requests.get(url).json()

def fetch_fixtures():
    url = "https://fantasy.premierleague.com/api/fixtures/"
    return pd.DataFrame(requests.get(url).json())

def get_current_pl_team_ids(data):
    return set(team['id'] for team in data['teams'])

Whitelist (top 5 scorers 24-25): {'Isak', 'Palmer', 'Mbeumo', 'Wood', 'M.Salah'}


In [ ]:
def is_likely_starter(player):
    minutes = player.get('minutes', 0)
    selected = float(player.get('selected_by_percent', '0').replace('%', '')) if 'selected_by_percent' in player else 0
    status = player.get('status', 'a')
    name = player.get('web_name', '')
    if status != 'a' and name not in WHITELIST:
        return False
    if minutes >= 1500:
        return True
    if selected >= 5 and status == 'a':
        return True
    if name in WHITELIST:
        return True
    return False

In [ ]:
def predict_points_preseason(players_df, fixtures_df, teams_data, lookahead=5):
    team_strength = {}
    for team in teams_data:
        if 'strength' in team:
            team_strength[team['id']] = team['strength']
        elif 'points' in team:
            team_strength[team['id']] = math.floor(5- (3*(team['position']-1)/19))
        else:
            team_strength[team['id']] = 2
    predicted_points_list = []
    for _, player in players_df.iterrows():
        if not is_likely_starter(player):
            predicted_points_list.append([0] * lookahead)
            continue
        base_points = player.get('total_points', 0)
        games_played = player.get('minutes', 0) / 90 if player.get('minutes', 0) else 1
        base_ppg = base_points / games_played if games_played > 0 else 2.5
        pos = int(player['element_type'])
        def_mult = 1.0
        if pos == 2:
            def_mult = 1.2
        elif pos == 3:
            def_mult = 1.1
        player_fixtures = fixtures_df[
            ((fixtures_df['team_h'] == player['team']) | (fixtures_df['team_a'] == player['team'])) &
            (fixtures_df['event'].notnull())
        ].sort_values('event').head(lookahead)
        gw_points = []
        for gw in range(1, lookahead + 1):
            fixture = player_fixtures[player_fixtures['event'] == gw]
            if fixture.empty:
                gw_points.append(0)
                continue
            fix = fixture.iloc[0]
            if fix['team_h'] == player['team']:
                opp = fix['team_a']
                difficulty = fix['team_h_difficulty']
            else:
                opp = fix['team_h']
                difficulty = fix['team_a_difficulty']
            player_team_strength = team_strength.get(player['team'], 10)
            opp_team_strength = team_strength.get(opp, 10)
            strength_factor = player_team_strength / (opp_team_strength + 1e-5)
            strength_factor = max(0.5, min(1.5, strength_factor))
            ep = base_ppg * def_mult * (6 - difficulty) / 5 * strength_factor
            gw_points.append(ep)
        predicted_points_list.append(gw_points)

    players_df['predicted_points_per_gw'] = predicted_points_list
    players_df['predicted_points'] = players_df['predicted_points_per_gw'].apply(sum)
    return players_df

In [ ]:
def factor_weight(current_gw):
    w_form = min(0.4, (current_gw-1)/4 * 0.4) if current_gw >= 1 else 0.0
    w_last = max(0.0, 0.6 * (1 - (current_gw-1)/37))
    w_current = min(0.2, 0.2 * (current_gw-1)/37)
    w_fixture = 0.4
    total = w_form + w_last + w_current + w_fixture
    w_form /= total
    w_last /= total
    w_current /= total
    w_fixture /= total
    return w_last, w_current, w_form, w_fixture

def last5_form(points_list, current_gw):
    if current_gw <= 1 or not points_list:
        return 0.0
    start = max(0, current_gw - 5)
    window = points_list[start:current_gw-1]
    if not window:
        return 0.0
    weights = np.linspace(0.5, 1.0, len(window))
    return float(np.average(window, weights=weights))

def predict_points_future_custom(player, current_gw, fixtures_df, teams_data, lookahead=5):
    games_last = player.get('minutes_last_season', 0) / 90 if player.get('minutes_last_season', 0) else 1
    ppg_last = player.get('total_points_last_season', 0) / games_last
    games_current = player.get('minutes_current', 0) / 90 if player.get('minutes_current', 0) else 1
    ppg_current = player.get('total_points_current', 0) / games_current
    points_list = player.get('recent_points', [])
    form_ppg = last5_form(points_list, current_gw)
    w_last, w_current, w_form, w_fixture = factor_weight(current_gw)
    combined_ppg = w_last * ppg_last + w_current * ppg_current + w_form * form_ppg
    team_strength = {team['id']: team.get('strength', 10) for team in teams_data}
    predicted_points = []
    for gw_offset in range(lookahead):
        gw = current_gw + gw_offset
        player_fixtures = fixtures_df[
            ((fixtures_df['team_h'] == player['team']) | (fixtures_df['team_a'] == player['team'])) &
            (fixtures_df['event'] == gw)
        ]
        if player_fixtures.empty:
            predicted_points.append(0)
            continue
        fix = player_fixtures.iloc[0]
        if fix['team_h'] == player['team']:
            opp = fix['team_a']
            difficulty = fix['team_h_difficulty']
        else:
            opp = fix['team_h']
            difficulty = fix['team_a_difficulty']
        team_str = team_strength.get(player['team'], 10)
        opp_str = team_strength.get(opp, 10)
        strength_factor = np.clip(team_str / (opp_str + 1e-5), 0.5, 1.5)
        fixture_factor = (6 - difficulty) / 5
        ep = combined_ppg * (w_fixture + (1 - w_fixture) * fixture_factor) * strength_factor
        predicted_points.append(ep)
    return predicted_points

def suggest_transfers(current_squad, all_players, current_gw, fixtures_df, teams_data, max_transfers=5, transfer_cost=4):
    predicted_points = {}
    for p in all_players:
        predicted_points[p['id']] = predict_points_future_custom(p, current_gw, fixtures_df, teams_data)[0]
    swaps = []
    for player_out in current_squad:
        for player_in in all_players:
            if player_in['id'] in [pl['id'] for pl in current_squad]:
                continue
            gain = predicted_points[player_in['id']] - predicted_points[player_out['id']] - transfer_cost
            if gain > 0:
                swaps.append({'out': player_out, 'in': player_in, 'gain': gain})
    swaps.sort(key=lambda x: x['gain'], reverse=True)
    return swaps[:max_transfers]

def select_captains(starting_xi, current_gw, fixtures_df, teams_data):
    predicted = {}
    for p in starting_xi:
        predicted[p['id']] = predict_points_future_custom(p, current_gw, fixtures_df, teams_data)[0]
    sorted_players = sorted(starting_xi, key=lambda x: predicted[x['id']], reverse=True)
    captain = sorted_players[0]
    vice_captain = sorted_players[1]
    return captain, vice_captain

def chip_recommendation(squad, current_gw, fixtures_df, teams_data):
    bench_points = [predict_points_future_custom(p, current_gw, fixtures_df, teams_data)[0] for p in squad[11:]]
    bench_boost_suggested = np.mean(bench_points) > 5
    captain_points = predict_points_future_custom(squad[0], current_gw, fixtures_df, teams_data)[0]
    triple_captain_suggested = captain_points > 8
    squad_points = sum([predict_points_future_custom(p, current_gw, fixtures_df, teams_data)[0] for p in squad])
    free_hit_suggested = squad_points < 50
    return {
        'bench_boost': bench_boost_suggested,
        'triple_captain': triple_captain_suggested,
        'free_hit': free_hit_suggested
    }


In [ ]:
def optimize_squad(players_df, budget=1000):
    players_df = players_df.reset_index(drop=True)
    prob = pulp.LpProblem("FPL_Squad_Selection", pulp.LpMaximize)
    x = [pulp.LpVariable(f"x{i}", cat="Binary") for i in range(len(players_df))]
    prob += pulp.lpSum([x[i] * players_df.loc[i, 'predicted_points'] for i in range(len(players_df))])
    prob += pulp.lpSum(x) == 15
    prob += pulp.lpSum([x[i] for i in range(len(players_df)) if players_df.loc[i, 'element_type'] == 1]) == 2  # GK
    prob += pulp.lpSum([x[i] for i in range(len(players_df)) if players_df.loc[i, 'element_type'] == 2]) == 5  # DEF
    prob += pulp.lpSum([x[i] for i in range(len(players_df)) if players_df.loc[i, 'element_type'] == 3]) == 5  # MID
    prob += pulp.lpSum([x[i] for i in range(len(players_df)) if players_df.loc[i, 'element_type'] == 4]) == 3  # FWD
    teams = players_df['team'].unique()
    for team in teams:
        prob += pulp.lpSum([x[i] for i in range(len(players_df)) if players_df.loc[i, 'team'] == team]) <= 3
    prob += pulp.lpSum([x[i] * players_df.loc[i, 'now_cost'] for i in range(len(players_df))]) <= budget
    solver = pulp.PULP_CBC_CMD(msg=1)
    prob.solve(solver)
    selected_indices = [i for i in range(len(players_df)) if pulp.value(x[i]) == 1]
    return players_df.loc[selected_indices]

In [ ]:
def build_starting_lineups(squad, fixtures_df, lookahead=5):
    rotations = {}
    per_gw_points = {}
    for _, player in squad.iterrows():
        per_gw_points[player['id']] = player['predicted_points_per_gw']
    for gw in range(lookahead):
        squad['gw_predicted_points'] = squad['id'].apply(lambda pid: per_gw_points[pid][gw])
        squad_sorted = squad.sort_values(by='gw_predicted_points', ascending=False)
        starting_xi = squad_sorted.head(11).copy()
        count_gk = sum(starting_xi['element_type'] == 1)
        count_def = sum(starting_xi['element_type'] == 2)
        count_mid = sum(starting_xi['element_type'] == 3)
        count_fwd = sum(starting_xi['element_type'] == 4)

        def swap_in_player(pos_needed, exclude_pos):
            nonlocal starting_xi, squad_sorted
            candidates_in = squad_sorted[(squad_sorted['element_type'] == pos_needed) & (~squad_sorted.index.isin(starting_xi.index))]
            if candidates_in.empty:
                return False
            player_in = candidates_in.iloc[0]

            removable = starting_xi[~starting_xi['element_type'].isin(exclude_pos)].sort_values(by='gw_predicted_points')
            if removable.empty:
                return False
            player_out = removable.iloc[0]

            starting_xi = starting_xi.drop(player_out.name)
            starting_xi = pd.concat([starting_xi, player_in.to_frame().T])
            return True
        if count_gk == 0:
            swap_in_player(1, exclude_pos=[])
        elif count_gk > 1:
            gks = starting_xi[starting_xi['element_type'] == 1].sort_values(by='gw_predicted_points')
            to_drop = gks.iloc[:-1]
            starting_xi = starting_xi.drop(to_drop.index)
        count_gk = sum(starting_xi['element_type'] == 1)
        count_def = sum(starting_xi['element_type'] == 2)
        count_mid = sum(starting_xi['element_type'] == 3)
        count_fwd = sum(starting_xi['element_type'] == 4)
        while count_def < 3:
            if not swap_in_player(2, exclude_pos=[1, 2]):
                break
            count_def += 1
            count_mid = sum(starting_xi['element_type'] == 3)
            count_fwd = sum(starting_xi['element_type'] == 4)
        while count_mid < 3:
            if not swap_in_player(3, exclude_pos=[1, 2, 3]):
                break
            count_mid += 1
            count_def = sum(starting_xi['element_type'] == 2)
            count_fwd = sum(starting_xi['element_type'] == 4)
        while count_fwd < 2:
            if not swap_in_player(4, exclude_pos=[1, 2, 3, 4]):
                break
            count_fwd += 1
            count_def = sum(starting_xi['element_type'] == 2)
            count_mid = sum(starting_xi['element_type'] == 3)
        if len(starting_xi) < 11:
            needed = 11 - len(starting_xi)
            extras = squad_sorted[~squad_sorted.index.isin(starting_xi.index)].head(needed)
            starting_xi = pd.concat([starting_xi, extras])
        if len(starting_xi) > 11:
            starting_xi = starting_xi.head(11)
        subs = squad_sorted[~squad_sorted.index.isin(starting_xi.index)].head(4)
        rotations[gw + 1] = {
            'starting_xi': starting_xi.sort_values(by='gw_predicted_points', ascending=False),
            'subs': subs
        }
    return rotations

In [ ]:
def print_rotation_plan(rotations, fixtures_df):
    pos_map = {1: 'GK', 2: 'DEF', 3: 'MID', 4: 'FWD'}
    pos_order = [4, 3, 2, 1]
    for gw, lineup in rotations.items():
        print(f"Gameweek {gw}:")
        starters = lineup['starting_xi'].copy()
        if len(starters) != 11:
            print(f"Warning: Starting XI size is {len(starters)} players, expected 11.")
        fixtures_gw = fixtures_df[fixtures_df['event'] == gw]

        def get_fixture_info(player):
            fix = fixtures_gw[(fixtures_gw['team_h'] == player['team']) | (fixtures_gw['team_a'] == player['team'])]
            if fix.empty:
                return ("No Fixture", None)
            fix = fix.iloc[0]
            opp = fix['team_a'] if fix['team_h'] == player['team'] else fix['team_h']
            diff = fix['team_h_difficulty'] if fix['team_h'] == player['team'] else fix['team_a_difficulty']
            return (opp, diff)
        starters['next_fixture'], starters['difficulty'] = zip(*starters.apply(get_fixture_info, axis=1))
        starters['pos_name'] = starters.apply(lambda r: f"{pos_map[r['element_type']]} {r['web_name']}", axis=1)
        starters['pos_order'] = starters['element_type'].apply(lambda x: pos_order.index(x))
        starters = starters.sort_values(by='pos_order')
        print(" Starting XI:")
        print(starters[['pos_name', 'team', 'gw_predicted_points', 'next_fixture', 'difficulty']].to_string(index=False))
        subs = lineup['subs'].copy()
        if len(subs) != 4:
            print(f"Warning: Subs size is {len(subs)} players, expected 4.")
        subs['next_fixture'], subs['difficulty'] = zip(*subs.apply(get_fixture_info, axis=1))
        subs['pos_name'] = subs.apply(lambda r: f"{pos_map[r['element_type']]} {r['web_name']}", axis=1)
        subs['pos_order'] = subs['element_type'].apply(lambda x: pos_order.index(x))
        subs = subs.sort_values(by='pos_order')
        print(" Subs:")
        print(subs[['pos_name', 'team', 'gw_predicted_points', 'next_fixture', 'difficulty']].to_string(index=False))
        print("\n")

In [ ]:
lookahead=5
data=fetch_fpl_data()
elements=pd.DataFrame(data['elements'])
fixtures=fetch_fixtures()
pl_team_ids=get_current_pl_team_ids(data)
players=elements[elements['team'].isin(pl_team_ids)][['id','web_name','team','now_cost','element_type','minutes','total_points','selected_by_percent','status']]
players=predict_points_preseason(players,fixtures,data['teams'],lookahead)
squad=optimize_squad(players,budget=1000)
rotations=build_starting_lineups(squad,fixtures,lookahead)
print("Final squad (15 players):")
print(squad[['web_name','element_type','team','now_cost','predicted_points']])
print("\nRotation plan for next 5 GWs:\n")
print_rotation_plan(rotations,fixtures)
transfers=suggest_transfers(current_squad=squad.to_dict('records'),all_players=players.to_dict('records'),current_gw=1,fixtures_df=fixtures,teams_data=data['teams'],max_transfers=5,transfer_cost=4)
print("\nSuggested transfers (max 5, -4 each):")
for t in transfers:
    print(f"Out: {t['out']['web_name']} | In: {t['in']['web_name']} | Expected Gain: {t['gain']:.2f}")
captain,vice_captain=select_captains(starting_xi=squad.head(11).to_dict('records'),current_gw=1,fixtures_df=fixtures,teams_data=data['teams'])
print(f"\nCaptain: {captain['web_name']}, Vice-Captain: {vice_captain['web_name']}")
chips=chip_recommendation(squad=squad.to_dict('records'),current_gw=1,fixtures_df=fixtures,teams_data=data['teams'])
print("\nChip Recommendations:")
for chip,suggested in chips.items():
    print(f"{chip}: {'Yes' if suggested else 'No'}")


Final squad (15 players):
       web_name  element_type  team  now_cost  predicted_points
229       James             2     7        55         23.541265
230    Chalobah             2     7        50         21.764478
252  João Pedro             4     7        75         24.863226
315        Beto             4     9        55         16.061475
391     M.Salah             3    12       145         43.402701
394       Gakpo             3    12        75         28.143607
397  Szoboszlai             3    12        65         24.497008
411  Ederson M.             1    13        55         15.501674
415    Gvardiol             2    13        60         18.147104
419  Matheus N.             2    13        55         18.857903
485        Hall             2    15        55         19.736117
499      Barnes             3    15        65         28.894782
501      J.Murphy           3    15        65         26.190401
515        Sels             1    16        50         13.815737
538        Woo